In [1]:
import numpy as np
import pandas as pd
import re
import string

import nltk
from nltk.corpus import stopwords
# from nltk.stem.porter import PorterStemmer
# from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer
nltk.download("stopwords")
nltk.download('wordnet')

import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer 
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB, ComplementNB, MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/ivankwok/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/ivankwok/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
amazondf = pd.read_csv("amazon.csv",)
amazondf

,product_id,product_name,category,discounted_price,actual_price,discount_percentage,rating,rating_count,about_product,user_id,user_name,review_id,review_title,review_content,img_link,product_link
0,B07JW9H4J1,Wayona Nylon Braided USB to Lightning Fast Cha...,Computers&Accessories|Accessories&Peripherals|...,₹399,"₹1,099",64%,4.2,"24,269",High Compatibility : Compatible With iPhone 12...,"AG3D6O4STAQKAY2UVGEUV46KN35Q,AHMY5CWJMMK5BJRBB...","Manav,Adarsh gupta,Sundeep,S.Sayeed Ahmed,jasp...","R3HXWT0LRP0NMF,R2AJM3LFTLZHFO,R6AQJGUP6P86,R1K...","Satisfied,Charging is really fast,Value for mo...",Looks durable Charging is fine tooNo complains...,https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Wayona-Braided-WN3LG1-Sy...
1,B098NS6PVG,Ambrane Unbreakable 60W / 3A Fast Charging 1.5...,Computers&Accessories|Accessories&Peripherals|...,₹199,₹349,43%,4.0,"43,994","Compatible with all Type C enabled devices, be...","AECPFYFQVRUWC3KGNLJIOREFP5LQ,AGYYVPDD7YG7FYNBX...","ArdKn,Nirbhay kumar,Sagar Viswanathan,Asp,Plac...","RGIQEG07R9HS2,R1SMWZQ86XIN8U,R2J3Y1WL29GWDE,RY...","A Good Braided Cable for Your Type C Device,Go...",I ordered this cable to connect my phone to An...,https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Ambrane-Unbreakable-Char...
2,B096MSW6CT,Sounce Fast Phone Charging Cable & Data Sync U...,Computers&Accessories|Accessories&Peripherals|...,₹199,"₹1,899",90%,3.9,"7,928",【 Fast Charger& Data Sync】-With built-in safet...,"AGU3BBQ2V2DDAMOAKGFAWDDQ6QHA,AESFLDV2PT363T2AQ...","Kunal,Himanshu,viswanath,sai niharka,saqib mal...","R3J3EQQ9TZI5ZJ,R3E7WBGK7ID0KV,RWU79XKQ6I1QF,R2...","Good speed for earlier versions,Good Product,W...","Not quite durable and sturdy,https://m.media-a...",https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Sounce-iPhone-Charging-C...
3,B08HDJ86NZ,boAt Deuce USB 300 2 in 1 Type-C & Micro USB S...,Computers&Accessories|Accessories&Peripherals|...,₹329,₹699,53%,4.2,"94,363",The boAt Deuce USB 300 2 in 1 cable is compati...,"AEWAZDZZJLQUYVOVGBEUKSLXHQ5A,AG5HTSFRRE6NL3M5S...","Omkar dhale,JD,HEMALATHA,Ajwadh a.,amar singh ...","R3EEUZKKK9J36I,R3HJVYCLYOY554,REDECAZ7AMPQC,R1...","Good product,Good one,Nice,Really nice product...","Good product,long wire,Charges good,Nice,I bou...",https://m.media-amazon.com/images/I/41V5FtEWPk...,https://www.amazon.in/Deuce-300-Resistant-Tang...
4,B08CF3B7N1,Portronics Konnect L 1.2M Fast Charging 3A 8 P...,Computers&Accessories|Accessories&Peripherals|...,₹154,₹399,61%,4.2,"16,905",[CHARGE & SYNC FUNCTION]- This cable comes wit...,"AE3Q6KSUK5P75D5HFYHCRAOLODSA,AFUGIFH5ZAFXRDSZH...","rahuls6099,Swasat Borah,Ajay Wadke,Pranali,RVK...","R1BP4L2HH9TFUP,R16PVJEXKV6QZS,R2UPDB81N66T4P,R...","As good as original,Decent,Good one for second...","Bought this instead of original apple, does th...",https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Portronics-Konnect-POR-1...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1460,B08L7J3T31,Noir Aqua - 5pcs PP Spun Filter + 1 Spanner | ...,Home&Kitchen|Kitchen&HomeAppliances|WaterPurif...,₹379,₹919,59%,4,"1,090",SUPREME QUALITY 90 GRAM 3 LAYER THIK PP SPUN F...,"AHITFY6AHALOFOHOZEOC6XBP4FEA,AFRABBODZJZQB6Z4U...","Prabha ds,Raghuram bk,Real Deal,Amazon Custome...","R3G3XFHPBFF0E8,R3C0BZCD32EIGW,R2EBVBCN9QPD9R,R...","Received the product without spanner,Excellent...","I received product without spanner,Excellent p...",https://m.media-amazon.com/images/I/41fDdRtjfx...,https://www.amazon.in/Noir-Aqua-Spanner-Purifi...
1461,B01M6453MB,Prestige Delight PRWO Electric Rice Cooker (1 ...,Home&Kitchen|Kitchen&HomeAppliances|SmallKitch...,"₹2,280","₹3,045",25%,4.1,"4,118","230 Volts, 400 watts, 1 Year","AFG5FM3NEMOL6BNFRV2NK5FNJCHQ,AGEINTRN6Z563RMLH...","Manu Bhai,Naveenpittu,Evatira Sangma,JAGANNADH...","R3DDL2UPKQ2CK9,R2SYYU1OATVIU5,R1VM993161IYRW,R...","ok,everything was good couldn't return bcoz I ...","ok,got everything as mentioned but the me

In [3]:
amazondf["discounted_price"] = amazondf["discounted_price"].str.replace("₹", "")
amazondf["discounted_price"] = amazondf["discounted_price"].str.replace(",", "")
amazondf["discounted_price"] = amazondf["discounted_price"].astype("float32")

amazondf["actual_price"] = amazondf["actual_price"].str.replace("₹", "")
amazondf["actual_price"] = amazondf["actual_price"].str.replace(",", "")
amazondf["actual_price"] = amazondf["actual_price"].astype("float32")

amazondf.head()

,product_id,product_name,category,discounted_price,actual_price,discount_percentage,rating,rating_count,about_product,user_id,user_name,review_id,review_title,review_content,img_link,product_link
0,B07JW9H4J1,Wayona Nylon Braided USB to Lightning Fast Cha...,Computers&Accessories|Accessories&Peripherals|...,399.0,1099.0,64%,4.2,"24,269",High Compatibility : Compatible With iPhone 12...,"AG3D6O4STAQKAY2UVGEUV46KN35Q,AHMY5CWJMMK5BJRBB...","Manav,Adarsh gupta,Sundeep,S.Sayeed Ahmed,jasp...","R3HXWT0LRP0NMF,R2AJM3LFTLZHFO,R6AQJGUP6P86,R1K...","Satisfied,Charging is really fast,Value for mo...",Looks durable Charging is fine tooNo complains...,https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Wayona-Braided-WN3LG1-Sy...
1,B098NS6PVG,Ambrane Unbreakable 60W / 3A Fast Charging 1.5...,Computers&Accessories|Accessories&Peripherals|...,199.0,349.0,43%,4.0,"43,994","Compatible with all Type C enabled devices, be...","AECPFYFQVRUWC3KGNLJIOREFP5LQ,AGYYVPDD7YG7FYNBX...","ArdKn,Nirbhay kumar,Sagar Viswanathan,Asp,Plac...","RGIQEG07R9HS2,R1SMWZQ86XIN8U,R2J3Y1WL29GWDE,RY...","A Good Braided Cable for Your Type C Device,Go...",I ordered this cable to connect my phone to An...,https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Ambrane-Unbreakable-Char...
2,B096MSW6CT,Sounce Fast Phone Charging Cable & Data Sync U...,Computers&Accessories|Accessories&Peripherals|...,199.0,1899.0,90%,3.9,"7,928",【 Fast Charger& Data Sync】-With built-in safet...,"AGU3BBQ2V2DDAMOAKGFAWDDQ6QHA,AESFLDV2PT363T2AQ...","Kunal,Himanshu,viswanath,sai niharka,saqib mal...","R3J3EQQ9TZI5ZJ,R3E7WBGK7ID0KV,RWU79XKQ6I1QF,R2...","Good speed for earlier versions,Good Product,W...","Not quite durable and sturdy,https://m.media-a...",https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Sounce-iPhone-Charging-C...
3,B08HDJ86NZ,boAt Deuce USB 300 2 in 1 Type-C & Micro USB S...,Computers&Accessories|Accessories&Peripherals|...,329.0,699.0,53%,4.2,"94,363",The boAt Deuce USB 300 2 in 1 cable is compati...,"AEWAZDZZJLQUYVOVGBEUKSLXHQ5A,AG5HTSFRRE6NL3M5S...","Omkar dhale,JD,HEMALATHA,Ajwadh a.,amar singh ...","R3EEUZKKK9J36I,R3HJVYCLYOY554,REDECAZ7AMPQC,R1...","Good product,Good one,Nice,Really nice product...","Good product,long wire,Charges good,Nice,I bou...",https://m.media-amazon.com/images/I/41V5FtEWPk...,https://www.amazon.in/Deuce-300-Resistant-Tang...
4,B08CF3B7N1,Portronics Konnect L 1.2M Fast Charging 3A 8 P...,Computers&Accessories|Accessories&Peripherals|...,154.0,399.0,61%,4.2,"16,905",[CHARGE & SYNC FUNCTION]- This cable comes wit...,"AE3Q6KSUK5P75D5HFYHCRAOLODSA,AFUGIFH5ZAFXRDSZH...","rahuls6099,Swasat Borah,Ajay Wadke,Pranali,RVK...","R1BP4L2HH9TFUP,R16PVJEXKV6QZS,R2UPDB81N66T4P,R...","As good as original,Decent,Good one for second...","Bought this instead of original apple, does th...",https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Portronics-Konnect-POR-1...


In [4]:
amazondf.drop_duplicates(inplace = True)

In [5]:
pricedf = amazondf[["product_id", "product_name", "category", "about_product", "discounted_price", "actual_price"]].copy()
pricedf.head()

,product_id,product_name,category,about_product,discounted_price,actual_price
0,B07JW9H4J1,Wayona Nylon Braided USB to Lightning Fast Cha...,Computers&Accessories|Accessories&Peripherals|...,High Compatibility : Compatible With iPhone 12...,399.0,1099.0
1,B098NS6PVG,Ambrane Unbreakable 60W / 3A Fast Charging 1.5...,Computers&Accessories|Accessories&Peripherals|...,"Compatible with all Type C enabled devices, be...",199.0,349.0
2,B096MSW6CT,Sounce Fast Phone Charging Cable & Data Sync U...,Computers&Accessories|Accessories&Peripherals|...,【 Fast Charger& Data Sync】-With built-in safet...,199.0,1899.0
3,B08HDJ86NZ,boAt Deuce USB 300 2 in 1 Type-C & Micro USB S...,Computers&Accessories|Accessories&Peripherals|...,The boAt Deuce USB 300 2 in 1 cable is compati...,329.0,699.0
4,B08CF3B7N1,Portronics Konnect L 1.2M Fast Charging 3A 8 P...,Computers&Accessories|Accessories&Peripherals|...,[CHARGE & SYNC FUNCTION]- This cable comes wit...,154.0,399.0


In [6]:
pricedf['category'] = pricedf['category'].apply(lambda x: " ".join(x.split('|')))
pricedf.head()

,product_id,product_name,category,about_product,discounted_price,actual_price
0,B07JW9H4J1,Wayona Nylon Braided USB to Lightning Fast Cha...,Computers&Accessories Accessories&Peripherals ...,High Compatibility : Compatible With iPhone 12...,399.0,1099.0
1,B098NS6PVG,Ambrane Unbreakable 60W / 3A Fast Charging 1.5...,Computers&Accessories Accessories&Peripherals ...,"Compatible with all Type C enabled devices, be...",199.0,349.0
2,B096MSW6CT,Sounce Fast Phone Charging Cable & Data Sync U...,Computers&Accessories Accessories&Peripherals ...,【 Fast Charger& Data Sync】-With built-in safet...,199.0,1899.0
3,B08HDJ86NZ,boAt Deuce USB 300 2 in 1 Type-C & Micro USB S...,Computers&Accessories Accessories&Peripherals ...,The boAt Deuce USB 300 2 in 1 cable is compati...,329.0,699.0
4,B08CF3B7N1,Portronics Konnect L 1.2M Fast Charging 3A 8 P...,Computers&Accessories Accessories&Peripherals ...,[CHARGE & SYNC FUNCTION]- This cable comes wit...,154.0,399.0


In [7]:
pricedf["description"] = pricedf["product_name"] + " " + pricedf["about_product"] + " " + pricedf["category"]
pricedf.head()

,product_id,product_name,category,about_product,discounted_price,actual_price,description
0,B07JW9H4J1,Wayona Nylon Braided USB to Lightning Fast Cha...,Computers&Accessories Accessories&Peripherals ...,High Compatibility : Compatible With iPhone 12...,399.0,1099.0,Wayona Nylon Braided USB to Lightning Fast Cha...
1,B098NS6PVG,Ambrane Unbreakable 60W / 3A Fast Charging 1.5...,Computers&Accessories Accessories&Peripherals ...,"Compatible with all Type C enabled devices, be...",199.0,349.0,Ambrane Unbreakable 60W / 3A Fast Charging 1.5...
2,B096MSW6CT,Sounce Fast Phone Charging Cable & Data Sync U...,Computers&Accessories Accessories&Peripherals ...,【 Fast Charger& Data Sync】-With built-in safet...,199.0,1899.0,Sounce Fast Phone Charging Cable & Data Sync U...
3,B08HDJ86NZ,boAt Deuce USB 300 2 in 1 Type-C & Micro USB S...,Computers&Accessories Accessories&Peripherals ...,The boAt Deuce USB 300 2 in 1 cable is compati...,329.0,699.0,boAt Deuce USB 300 2 in 1 Type-C & Micro USB S...
4,B08CF3B7N1,Portronics Konnect L 1.2M Fast Charging 3A 8 P...,Computers&Accessories Accessories&Peripherals ...,[CHARGE & SYNC FUNCTION]- This cable comes wit...,154.0,399.0,Portronics Konnect L 1.2M Fast Charging 3A 8 P...


In [8]:
def remove_stopwords(sentence):
    stop = set(stopwords.words('english'))
    processed = " ".join([word for word in sentence.split() if word not in stop])
    return processed

def remove_urls(sentence):
    processed = re.sub('http[s]?://(?:[a-zA-Z]|[0–9]|[$-_@.&+#]|[!*\(\),]|'\
                   '(?:%[0–9a-fA-F][0–9a-fA-F]))+','', sentence)
    return processed

def remove_punctuation(sentence):
    punc = string.punctuation
    sentence = sentence.translate(str.maketrans('', '', punc))
    return sentence


In [9]:
pricedf["description"] = pricedf["description"].apply(lambda x: remove_urls(x))

pricedf["description"] = pricedf["description"].apply(lambda x: x.lower())

pricedf["description"] = pricedf["description"].apply(lambda x: remove_stopwords(x))

pricedf["description"] = pricedf["description"].apply(lambda x: remove_punctuation(x))

pricedf.head()

,product_id,product_name,category,about_product,discounted_price,actual_price,description
0,B07JW9H4J1,Wayona Nylon Braided USB to Lightning Fast Cha...,Computers&Accessories Accessories&Peripherals ...,High Compatibility : Compatible With iPhone 12...,399.0,1099.0,wayona nylon braided usb lightning fast chargi...
1,B098NS6PVG,Ambrane Unbreakable 60W / 3A Fast Charging 1.5...,Computers&Accessories Accessories&Peripherals ...,"Compatible with all Type C enabled devices, be...",199.0,349.0,ambrane unbreakable 60w 3a fast charging 15m ...
2,B096MSW6CT,Sounce Fast Phone Charging Cable & Data Sync U...,Computers&Accessories Accessories&Peripherals ...,【 Fast Charger& Data Sync】-With built-in safet...,199.0,1899.0,sounce fast phone charging cable data sync us...
3,B08HDJ86NZ,boAt Deuce USB 300 2 in 1 Type-C & Micro USB S...,Computers&Accessories Accessories&Peripherals ...,The boAt Deuce USB 300 2 in 1 cable is compati...,329.0,699.0,boat deuce usb 300 2 1 typec micro usb stress...
4,B08CF3B7N1,Portronics Konnect L 1.2M Fast Charging 3A 8 P...,Computers&Accessories Accessories&Peripherals ...,[CHARGE & SYNC FUNCTION]- This cable comes wit...,154.0,399.0,portronics konnect l 12m fast charging 3a 8 pi...


In [10]:
lemmatizer = WordNetLemmatizer()
def lemmatization(sentence):
    sentence = ' '.join([lemmatizer.lemmatize(word) for word in sentence.split()])
    return sentence

pricedf["description"] = pricedf["description"].apply(lambda x: lemmatization(x))
pricedf.head()

,product_id,product_name,category,about_product,discounted_price,actual_price,description
0,B07JW9H4J1,Wayona Nylon Braided USB to Lightning Fast Cha...,Computers&Accessories Accessories&Peripherals ...,High Compatibility : Compatible With iPhone 12...,399.0,1099.0,wayona nylon braided usb lightning fast chargi...
1,B098NS6PVG,Ambrane Unbreakable 60W / 3A Fast Charging 1.5...,Computers&Accessories Accessories&Peripherals ...,"Compatible with all Type C enabled devices, be...",199.0,349.0,ambrane unbreakable 60w 3a fast charging 15m b...
2,B096MSW6CT,Sounce Fast Phone Charging Cable & Data Sync U...,Computers&Accessories Accessories&Peripherals ...,【 Fast Charger& Data Sync】-With built-in safet...,199.0,1899.0,sounce fast phone charging cable data sync usb...
3,B08HDJ86NZ,boAt Deuce USB 300 2 in 1 Type-C & Micro USB S...,Computers&Accessories Accessories&Peripherals ...,The boAt Deuce USB 300 2 in 1 cable is compati...,329.0,699.0,boat deuce usb 300 2 1 typec micro usb stress ...
4,B08CF3B7N1,Portronics Konnect L 1.2M Fast Charging 3A 8 P...,Computers&Accessories Accessories&Peripherals ...,[CHARGE & SYNC FUNCTION]- This cable comes wit...,154.0,399.0,portronics konnect l 12m fast charging 3a 8 pi...


In [11]:
X_train, X_test, y_train, y_test = train_test_split(pricedf["description"], pricedf["actual_price"],  test_size=0.2, random_state=11, shuffle=True, stratify=None)

In [12]:
tv = TfidfVectorizer()
# train_tfidf = tv.fit_transform(ratingdf["combined_review"]).toarray()
X_train_tfidf = tv.fit_transform(X_train) # fit train
X_test_tfidf = tv.transform(X_test) # transform test
# train_tfidf = pd.DataFrame(train_tfidf, columns = tv.get_feature_names_out(), index = traindf.index)
# train_tfidf["rating_adj"]  = traindf["rating_adj"]
X_train_tfidf.shape, X_test_tfidf.shape

((1172, 14294), (293, 14294))

Linear Regression

In [13]:
from sklearn.linear_model import LinearRegression
linear = LinearRegression()
linear.fit(X_train_tfidf, y_train)
print(linear.score(X_test_tfidf, y_test))

0.8352586928558817


Support Vector Regression

In [14]:
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
sc = StandardScaler()
# X_train_sc = sc.fit_transform(X_train_tfidf)
# y_train_sc = sc.fit_transform(y_train)
# X_test_sc = sc.transform(X_test_tfidf)
# y_test_sc = sc.transform(y_test)

svr = SVR(kernel='linear')
svr.fit(X_train_tfidf, y_train)
print(svr.score(X_test_tfidf, y_test))

-0.12826948296063856


In [15]:
from sklearn.linear_model import Ridge
ridge = Ridge(alpha = 0.5)
ridge.fit(X_train_tfidf, y_train)
print(ridge.score(X_test_tfidf, y_test))

0.8561999977839305
